## Summaries for SCADS Application 

Utilizing Occams to build single document summaries for SCADS Demo application use 

In [1]:
import pandas as pd
import sqlite3
import multisum as msu
import re
from tqdm import tqdm

### Access Database

In [2]:
# get data from sqlite file
# resources: https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html
con = sqlite3.connect("data_large/news (1).sqlite")

# use this to check schema details and table names
# cur = con.cursor()
# cur.execute('SELECT * FROM sqlite_master')
# cur.fetchall()

df = pd.read_sql_query("SELECT * FROM article", con)
con.close()

In [3]:
df.shape

(22771, 6)

In [4]:
# Note: The database does not have the text from the article... boo. Merging values based on nid together.
# This dataset has scrapped the article text from the URL in 'efs/home/strotto/MIND HTML Pre-Processing.ipynb'
MIND_w_topics=pd.read_csv("~/efs/home/pcorona_content/MIND_Train_w_Topics.csv")
MIND_w_topics.sort_values(by=['Title', '_score_'], ascending=False, inplace=True)
MIND_w_topics.drop_duplicates(subset=['Title'], keep='first', inplace=True)

In [5]:
dff = df.merge(MIND_w_topics, how='left', left_on='title', right_on='Title')

In [6]:
to_sum = dff[~dff['Text'].isna()].reset_index()[['nid', 'cat', 'subcat', 'Title', 'abstract', 'display_date', 'Text']]
to_sum['summary'] = ''
to_sum

,nid,cat,subcat,Title,abstract,display_date,Text,summary
0,N61407,news,newspolitics,"Giuliani associates: Debts, lawsuits and a hig...",As federal prosecutors pursue their case again...,11/09/2019,As federal prosecutors pursue their case again...,
1,N31021,foodanddrink,newstrends,A Thai Soup Pop-Up Will Make a Fall Comeback a...,"Plus, the Royal is all about Clue cocktails ne...",11/09/2019,"Pop-ups seem to be everywhere in D.C. Here, Ea...",
2,N22152,news,newspolitics,Joe Biden To Campaign In Los Angeles Next Week,Former Vice President Joe Biden will speak a L...,11/09/2019,"LOS ANGELES, CA — Former Vice President Joe Bi...",
3,N31914,travel,travelarticle,"Voted 'Best Theme Park Christmas Event,' Dolly...","Get your jingle bells ready, because the Best ...",11/09/2019,"Get your jingle bells ready, because the Best ...",
4,N2142,news,newsgoodnews,Young trick-or-treater refills stranger's empt...,The touching moment was captured on a resident...,11/09/2019,While many kids dressed up in costumes to go t...,
...,...,...,...,...,...,...,...,...
17427,N19321,news,newspolitics,Most Pennsylvania voters have dim view of Trum...,One year before Pennsylvania voters will head ...,11/15/2019,One year before Pennsylvania voters will head ...,
17428,N29533,sports,tennis,Djokovic's hopes crushed as Federer serves up ...,Djokovic's hopes crushed as Federer serves up ...,11/15/2019,By Martyn Herman LONDON (Reuters) - Roger Fede...,
17429,N10522,sports,football_nfl,"Patrick Peterson will play vs. 49ers, per Card...",,11/15/2019,Arizona Cardinals cornerback Patrick Peterson ...,
17430,N47370,sports,football_nfl,4 things to watch for the rest of this Detroit...,Looking for a reason to keep watching this foo...,11/15/2019,Welcome to the end of hope! After last week’s ...,


### Summarize

In [7]:
# loop through all texts to summarize 
for ele in tqdm(range(0, len(to_sum))):
    documents, doc_incidences, load_time = msu.load_corpus(to_sum.loc[ele:ele].reset_index(), 'Text')
    # preprocessing occurs here, removing starting phrases and URLs
    a = [x.text for x in documents[0].sentences if not bool(re.match("[A-Z]+( |:|-){1}", str(x.text))) and not bool(re.match('(.*)(\.com|\.edu|\.net|\.org)', str(x.text)))]
    to_sum['Text'][ele] = " ".join(a)
    documents, doc_incidences, load_time = msu.load_corpus(to_sum.loc[ele:ele].reset_index(), 'Text')

    # if only 1 sentance has been recognized, then don't allow for summarization 
    if len(documents[0].sentences) > 1:
        # summarizing with scheme "sentences", of length "300"
        build_time, extract_time, sentences, doc_titles_new, extractor, sentence_weights = msu.summarize_collect(to_sum.loc[ele:ele].reset_index(), documents, doc_incidences, 'sentances', 300, False)
        summary_text = [sentences[x].text for x in range(0, len(sentences))]
    else: 
        summary_text = 'Summary Not Available'
        
    to_sum['summary'][ele] = summary_text

to_sum.rename(columns = {'Title': 'title', 'Text':'text'}, inplace=True)
to_sum.to_csv("out/sumSCADS_v3.csv", index=False)

100%|██████████| 17432/17432 [14:57<00:00, 19.43it/s]


In [36]:
to_sum['summary'] = [None if x == '' or x == [] or x == 'Summary Not Available' else x for x in to_sum['summary']]

### Export data back to sqllite file

In [37]:
# keep the original data (even if it doesn't have a summary)
wdf = df.merge(to_sum[['nid', 'summary']], how='left', left_on='nid', right_on='nid')

In [71]:
# turn array into string w/ unique delimiter 
wdf['sum'] = [" | ".join(x) if type(x) == list else None for x in wdf['summary']]
wdf = wdf[['nid', 'cat', 'subcat', 'title', 'abstract', 'display_date', 'sum']]

In [79]:
con = sqlite3.connect("data_large/news (1).sqlite")

# Write the new DataFrame to a new SQLite table
wdf.to_sql("articlesSum", con, if_exists="replace", index=False)

con.close()